In [12]:
import os

import matplotlib.pyplot as plt
import numpy as np

"Machine learning tools"
import pickle

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold, train_test_split


from classification.datasets import Dataset
from classification.utils.audio_student import AudioUtil, Feature_vector_DS

from classification.utils.plots import (
    plot_decision_boundaries,
    plot_specgram,
    show_confusion_matrix,
)
from classification.utils.utils import accuracy

In [13]:
np.random.seed(0)

In [14]:
### TO RUN
dataset = Dataset()
classnames = dataset.list_classes()

print("\n".join(classnames))

chainsaw
fire
fireworks
gunshot


In [15]:
### TO RUN
fm_dir = "data/feature_matrices/"  # where to save the features matrices
new_dataset_dir = "src/classification/datasets/new_dataset/melvecs/"
model_dir = "data/models/xgboost"  # where to save the models
os.makedirs(fm_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [16]:
### TO RUN

"Creation of the dataset"
myds = Feature_vector_DS(dataset, Nft=512, nmel=20, duration=950, shift_pct=0.0)

"Some attributes..."
myds.nmel
myds.duration
myds.shift_pct
myds.sr
myds.data_aug
myds.ncol

idx = 0

In [17]:
# TRANSFORMATION ON FEATURE VECTOR

def add_noise(feature_vector, snr_db=15):
    """Adds white noise to a feature vector based on the given SNR (Signal-to-Noise Ratio)."""
    power_signal = np.mean(feature_vector ** 2)
    power_noise = power_signal / (10 ** (snr_db / 10))
    noise = np.random.normal(0, np.sqrt(power_noise), feature_vector.shape)
    return feature_vector + noise

def shifting(feature_vector, shift_max=20):
    """Shifts mel spectrogram feature vectors along the time axis by a random shift between 0 and shift_max."""
    shift = np.random.randint(0, shift_max)
    return np.roll(feature_vector, shift, axis=0)  # Rolling along the first axis

In [18]:

import numpy as np

train_pct = 0.7
data_aug_factor = 1
featveclen = len(myds["fire", 0, "", ""])  # Same for all classes
classnames = ["chainsaw", "fire", "fireworks", "gunshot"]  # Or wherever you store class names
nclass = len(classnames)

# Determine number of samples per class
naudio_per_class = {"chainsaw" : 76, "fire" : 76, "fireworks" : 76, "gunshot" : 40}


# Allocate feature matrix
total_samples_basic = sum(naudio_per_class[c] for c in classnames)
X_basic = np.zeros((total_samples_basic, featveclen))
y_basic = np.zeros((total_samples_basic), dtype=object)
total_samples_basic
# Fill feature matrix
idx = 0
for class_idx, classname in enumerate(classnames):
    for i in range(naudio_per_class[classname]):
        featvec = myds[classname, i, "", ""]
        X_basic[idx, :] = featvec
        y_basic[idx] = classname
        idx += 1

# Save features and labels
np.save(fm_dir + "X_basic.npy", X_basic)
np.save(fm_dir + "y_basic.npy", y_basic)

print(f"Shape of the basic feature matrix : {X_basic.shape}")
print(f"Number of labels : {y_basic.shape}")


Shape of the basic feature matrix : (268, 400)
Number of labels : (268,)


We can now create a new augmented dataset and observe if the classification results improve. 

In [19]:

### AUGMENTED DATASET
list_augmentation = ["original", "noise", "shifting"]
myds.mod_data_aug(list_augmentation)
print("Number of transformations : ", myds.data_aug_factor)


# Calcul total des échantillons
total_aug_samples = sum(naudio_per_class[c] for c in classnames) * len(list_augmentation)
X_basic_aug = np.zeros((total_aug_samples, featveclen))
y_basic_aug = np.zeros((total_aug_samples), dtype=object)

# Remplissage des features
idx = 0
for aug in list_augmentation:
    for classname in classnames:
        for i in range(naudio_per_class[classname]):
            featvec = myds[classname, i, aug, ""]
            X_basic_aug[idx, :] = featvec
            y_basic_aug[idx] = classname
            idx += 1

# Sauvegarde
np.save(fm_dir + "X_basic_aug.npy", X_basic_aug)
np.save(fm_dir + "y_basic_aug.npy", y_basic_aug)

print(f"Shape of the feature matrix : {X_basic_aug.shape}")
print(f"Number of labels : {y_basic_aug.shape}")
print(f"------------------------------------------------------------")
print(f"Transformations: {list_augmentation}. Labels aligned dynamically with class sizes.")


Number of transformations :  3
Shape of the feature matrix : (804, 400)
Number of labels : (804,)
------------------------------------------------------------
Transformations: ['original', 'noise', 'shifting']. Labels aligned dynamically with class sizes.


In [20]:
RUN = False

if RUN:
    import os
    import numpy as np
    import matplotlib.pyplot as plt
    from classification.utils.plots import plot_specgram_textlabel

    # Charger les données
    X = np.load(os.path.join(fm_dir, "X_basic_aug.npy"), allow_pickle=True)
    y = np.load(os.path.join(fm_dir, "y_basic_aug.npy"), allow_pickle=True)

    # Dossier où sauvegarder les images
    save_dir = os.path.join("src/classification/soundfiles_melspec_augmentation")
    os.makedirs(save_dir, exist_ok=True)

    # Nombre d'exemples de base (avant augmentation)
    length_X_basic = int(len(X) / len(list_augmentation))

    # Boucle de sauvegarde
    for i in range(length_X_basic):
        for j, aug_name in enumerate(list_augmentation):
            idx = i + j * length_X_basic
            melspec = X[idx]
            class_of_spec = y[idx]

            fig, ax = plt.subplots()
            plot_specgram_textlabel(
                melspec.reshape((20, 20)),
                ax=ax,
                is_mel=True,
                title=f"MEL Spectrogram #{i} - {aug_name}",
                xlabel="Mel vector",
                textlabel=f"{class_of_spec} (aug: {aug_name})",
            )
            plt.tight_layout()
            save_path = os.path.join(save_dir, f"melspec_{i}_{aug_name}.png")
            fig.savefig(save_path)
            plt.close(fig)


FINAL MODEL SAVE

In [ ]:

import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

TEST_SET = True

A = True # PCA NOAUG NONORM
B = True # NOPCA NOAUG NONORM
C = True # PCA AUG NONORM
D = True # NOPCA AUG NONORM
E = True # NOPCA NOAUG NONORM
F = True # PCA NOAUG NORM
G = True # PCA AUG NORM
H = True # NOPCA AUG NORM

# Load datasets
X_basic_aug = np.load(os.path.join(fm_dir, "X_basic_aug.npy"))
y_basic_aug = np.load(os.path.join(fm_dir, "y_basic_aug.npy"), allow_pickle=True)

X_basic = np.load(os.path.join(fm_dir, "X_basic.npy"))
y_basic = np.load(os.path.join(fm_dir, "y_basic.npy"), allow_pickle=True)

# Encode labels
label_encoder = LabelEncoder()
y_basic = label_encoder.fit_transform(y_basic)
y_basic_aug = label_encoder.transform(y_basic_aug)


# Split the dataset into training and testing sets
if TEST_SET:
    X_train, X_test, y_train, y_test = train_test_split(X_basic, y_basic, test_size=0.3, random_state=42)
    X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(X_basic_aug, y_basic_aug, test_size=0.3, random_state=42)
else:
    X_train = X_basic
    y_train = y_basic
    X_train_aug = X_basic_aug
    y_train_aug = y_basic_aug

# =========================
# SCENARIO A: WITH PCA (no aug)
# =========================
if A:
    pca = PCA(n_components=0.98)
    X_train_pca = pca.fit_transform(X_train)
    if TEST_SET:
        X_test_pca = pca.transform(X_test)

    pca_filename = os.path.join(model_dir, "pca_noaug_nonorm.pickle")
    with open(pca_filename, "wb") as f:
        pickle.dump(pca, f)

    xgb_pca = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                            subsample=0.8, colsample_bytree=0.8, eval_metric='mlogloss', random_state=42)
    xgb_pca.fit(X_train_pca, y_train)

    model_filename = os.path.join(model_dir, "xgb_pca_noaug_nonorm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_pca, f)

# =========================
# SCENARIO B: WITHOUT PCA (no aug)
# =========================
if B:
    xgb_no_pca = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                            subsample=0.8, colsample_bytree=0.8, eval_metric='mlogloss', random_state=42)
    xgb_no_pca.fit(X_train, y_train)

    model_filename = os.path.join(model_dir, "xgb_nopca_noaug_nonorm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_no_pca, f)

# =========================
# SCENARIO C: WITH PCA (aug)
# =========================
if C :
    pca = PCA(n_components=0.98)
    X_train_aug_pca = pca.fit_transform(X_train_aug)
    if TEST_SET:
        X_test_aug_pca = pca.transform(X_test_aug)

    pca_filename = os.path.join(model_dir, "pca_aug_nonorm.pickle")
    with open(pca_filename, "wb") as f:
        pickle.dump(pca, f)

    xgb_model_pca = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,
                                subsample=subsample, colsample_bytree=colsample_bytree,
                                eval_metric='mlogloss', random_state=42)
    xgb_model_pca.fit(X_train_aug_pca, y_train_aug)

    model_filename = os.path.join(model_dir, "xgb_pca_aug_nonorm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_model_pca, f)

# =========================
# SCENARIO D: WITHOUT PCA (aug)
# =========================
if D :
    xgb_model_no_pca = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,
                                    subsample=subsample, colsample_bytree=colsample_bytree,
                                    eval_metric='mlogloss', random_state=42)
    xgb_model_no_pca.fit(X_train_aug, y_train_aug)

    model_filename = os.path.join(model_dir, "xgb_nopca_aug_nonorm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_model_no_pca, f)

# =========================
# SCENARIO E: NO DATA TRANSFORMATION (no aug)
# =========================
if E :
    xgb_model_no_transform = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,
                                        subsample=subsample, colsample_bytree=colsample_bytree,
                                        eval_metric='mlogloss', random_state=42)
    xgb_model_no_transform.fit(X_train, y_train)

    model_filename = os.path.join(model_dir, "xgb_nopca_noaug_nonorm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_model_no_transform, f)

# =========================
# SCENARIO F: NORMALIZATION + PCA (no aug)
# =========================
if F :
    X_train_norm = np.array([x/np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X_train])
    if TEST_SET:
        X_test_norm = np.array([x/np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X_test])

    pca = PCA(n_components=0.98)
    X_train_norm_pca = pca.fit_transform(X_train_norm)
    if TEST_SET:
        X_test_norm_pca = pca.transform(X_test_norm)

    pca_filename = os.path.join(model_dir, "pca_noaug_norm.pickle")
    with open(pca_filename, "wb") as f:
        pickle.dump(pca, f)

    xgb_model_norm_pca = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,
                                    subsample=subsample, colsample_bytree=colsample_bytree,
                                    eval_metric='mlogloss', random_state=42)
    xgb_model_norm_pca.fit(X_train_norm_pca, y_train)

    model_filename = os.path.join(model_dir, "xgb_pca_noaug_norm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_model_norm_pca, f)

# =========================
# SCENARIO G: NORMALIZATION + AUG + PCA
# =========================
if G : 
    X_train_aug_norm = np.array([x/np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X_train_aug])
    if TEST_SET:
        X_test_aug_norm = np.array([x/np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X_test_aug])

    pca = PCA(n_components=0.98)
    X_train_aug_norm_pca = pca.fit_transform(X_train_aug_norm)
    if TEST_SET:
        X_test_aug_norm_pca = pca.transform(X_test_aug_norm)

    pca_filename = os.path.join(model_dir, "pca_aug_norm.pickle")
    with open(pca_filename, "wb") as f:
        pickle.dump(pca, f)

    xgb_model_norm_aug_pca = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,
                                        subsample=subsample, colsample_bytree=colsample_bytree,
                                        eval_metric='mlogloss', random_state=42)
    xgb_model_norm_aug_pca.fit(X_train_aug_norm_pca, y_train_aug)

    model_filename = os.path.join(model_dir, "xgb_pca_aug_norm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_model_norm_aug_pca, f)


# =========================
# SCENARIO H: NORMALIZATION + AUG (no PCA)
# =========================
if H:
    X_train_aug_norm = np.array([x / np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X_train_aug])
    if TEST_SET:
        X_test_aug_norm = np.array([x / np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X_test_aug])

    xgb_model_aug_norm = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                                       subsample=0.8, colsample_bytree=0.8,
                                       eval_metric='mlogloss', random_state=42)
    xgb_model_aug_norm.fit(X_train_aug_norm, y_train_aug)

    model_filename = os.path.join(model_dir, "xgb_nopca_aug_norm.pickle")
    with open(model_filename, "wb") as f:
        pickle.dump(xgb_model_aug_norm, f)


# =========================
# EVALUATION FUNCTION
# =========================
def evaluate_model(model, X_test, y_test, description):
    predict = model.predict(X_test)

    classes = np.unique(y_test)
    precision_per_class = precision_score(y_test, predict, average=None, labels=classes)
    recall_per_class = recall_score(y_test, predict, average=None, labels=classes)
    test_accuracy_per_class = []
    conf_matrix = confusion_matrix(y_test, predict, labels=classes)

    for i, cls in enumerate(classes):
        acc = conf_matrix[i, i] / conf_matrix[i, :].sum()
        test_accuracy_per_class.append(acc)

    cv_scores = cross_val_score(model, X_test, y_test, cv=5, scoring='accuracy')
    mean_cv_accuracy = np.mean(cv_scores)

    print(f"\n=== {description} ===")
    print(f"Test Accuracy (Overall): {np.mean(predict == y_test):.4f}")
    print(f"Mean CV Accuracy: {mean_cv_accuracy:.4f}")

    print("\nPer-Class Metrics:")
    for i, cls in enumerate(classes):
        print(f"Class {cls}: Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, Accuracy={test_accuracy_per_class[i]:.4f}")

# =========================
# EVALUATE ALL MODELS
# =========================
if TEST_SET:
    if A :
        evaluate_model(xgb_pca, X_test_pca, y_test, "Scenario A: PCA NOAUG NONORM")
    if B :
        evaluate_model(xgb_no_pca, X_test, y_test, "Scenario B: NOPCA NOAUG NONORM")
    if C:
        evaluate_model(xgb_model_pca, X_test_aug_pca, y_test_aug, "Scenario C: PCA AUG NONORM")
    if D:
        evaluate_model(xgb_model_no_pca, X_test, y_test, "Scenario D: NOPCA AUG NONORM")
    if E:
        evaluate_model(xgb_model_no_transform, X_test, y_test, "Scenario E: NOPCA NOAUG NONORM")
    if F:
        evaluate_model(xgb_model_norm_pca, X_test_norm_pca, y_test, "Scenario F: PCA NOAUG NORM")
    if G:
        evaluate_model(xgb_model_norm_aug_pca, X_test_aug_norm_pca, y_test_aug, "Scenario G: PCA AUG NORM")

2025-04-14 14:09:44.993239: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-14 14:09:44.995186: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-14 14:09:44.998363: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-14 14:09:45.004973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744632585.019196     852 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744632585.02

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

=== Scenario B: CNN NOPCA NOAUG NONORM ===
              precision    recall  f1-score   support

           0       0.58      0.83      0.68        23
           1       0.80      0.74      0.77        27
           2       0.79      0.61      0.69        18
           3       0.78      0.54      0.64        13

    accuracy                           0.70        81
   macro avg       0.73      0.68      0.69        81
weighted avg       0.73      0.70      0.70        81

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

=== Scenario D: CNN NOPCA AUG NONORM ===
              precision    recall  f1-score   support

           0       0.96      0.93      0.94        73
           1       0.94      0.92      0.93        65
           2       0.86      0.91      0.89        69
           3       0.91      0.89      0.90        35

    accuracy                           0.92       242
   macro avg       0.92      0.91      0.92       242
weighted avg       0.92

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

=== Scenario E: CNN NOPCA NOAUG NONORM ===
              precision    recall  f1-score   support

           0       0.86      0.78      0.82        23
           1       0.88      0.85      0.87        27
           2       0.75      0.83      0.79        18
           3       0.79      0.85      0.81        13

    accuracy                           0.83        81
   macro avg       0.82      0.83      0.82        81
weighted avg       0.83      0.83      0.83        81



8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

=== Scenario H: CNN NOPCA AUG NORM ===
              precision    recall  f1-score   support

           0       0.30      1.00      0.46        73
           1       0.00      0.00      0.00        65
           2       0.00      0.00      0.00        69
           3       0.00      0.00      0.00        35

    accuracy                           0.30       242
   macro avg       0.08      0.25      0.12       242
weighted avg       0.09      0.30      0.14       242



/home/ppasture/master_projet_elec/LELEC210X/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ppasture/master_projet_elec/LELEC210X/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ppasture/master_projet_elec/LELEC210X/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

HYPERPARAMETER TUNING

In [ ]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from bayes_opt import BayesianOptimization

# Your custom accuracy function
from classification.utils.utils import accuracy

# --- CONFIG FLAGS ---
NORMALIZATION = False
TRANSFORMATION = True

# --- STEP 1: Load/Select Data ---
if TRANSFORMATION:
    try:
        X = X_basic_aug   # Make sure X_basic_aug is defined in your environment
        y = y_basic_aug   # Make sure y_basic_aug is defined in your environment
    except NameError:
        raise ValueError("X_aug and y_aug must be defined before running this script.")
else:
    try:
        X = X_basic       # Make sure X_basic is defined in your environment
        y = y_basic       # Make sure y_basic is defined in your environment
    except NameError:
        raise ValueError("X and y must be defined before running this script.")

# Optional normalization
if NORMALIZATION:
    X = np.array([
        x / np.linalg.norm(x) if np.linalg.norm(x) != 0 else x
        for x in X
    ])

# --- STEP 2: Define the Objective Function for Bayesian Optimization ---
def xgb_cv(
    n_estimators,
    max_depth,
    learning_rate,
    subsample,
    colsample_bytree
):
    """
    This function trains an XGBClassifier with given hyperparameters
    and returns the mean CV accuracy as the objective to maximize.
    """
    # Convert some parameters to int, as required by XGBoost
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)

    model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, subsample=subsample, colsample_bytree=colsample_bytree,
        eval_metric='mlogloss',
        # remove use_label_encoder (deprecated)
        random_state=42
    )
    
    # 5-fold cross-validation on the *entire dataset* X, y
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    
    # Return the mean of cross-validation accuracy
    return cv_scores.mean()

# --- STEP 3: Set Up the Bayesian Optimizer ---
# Hyperparameter search space
pbounds = {
    'n_estimators': (50, 400),      # e.g. from 50 to 300
    'max_depth': (2, 15),           # integer between 2 and 12
    'learning_rate': (0.01, 0.3),   # from 0.01 to 0.3
    'subsample': (0.5, 1),        # from 0.5 to 1.0
    'colsample_bytree': (0.5, 1)  # from 0.5 to 1.0
}

optimizer = BayesianOptimization(
    f=xgb_cv,        # The function we want to maximize
    pbounds=pbounds, # The search space
    random_state=42  # Ensures reproducibility
)

# --- STEP 4: Run the Bayesian Optimization Loop ---
# We'll do a few initial random explorations (init_points) 
# and then a certain number of optimization steps (n_iter).
init_points = 3
n_iter = 20

print("Starting Bayesian Optimization...")
best_score_so_far = -1.0
early_stop_threshold = 0.90  # Stop if we exceed 90% cross-val accuracy

optimizer.maximize(init_points=init_points, n_iter=n_iter)

for i, res in enumerate(optimizer.res):
    score = res['target']
    print(f"Iteration {i+1}, CV Accuracy: {score:.4f}, Parameters: {res['params']}")
    
    if score > best_score_so_far:
        best_score_so_far = score
    
    # Early stopping if we found a "good" configuration
    if best_score_so_far > early_stop_threshold:
        print(f"\nEarly stopping: Found cross-validation accuracy above {early_stop_threshold}\n")
        break

# --- STEP 5: Get the Best Found Hyperparameters ---
best_params = optimizer.max['params']
best_n_estimators = int(best_params['n_estimators'])
best_max_depth = int(best_params['max_depth'])
best_learning_rate = best_params['learning_rate']
best_subsample = best_params['subsample']
best_colsample_bytree = best_params['colsample_bytree']

print("\n=== BEST HYPERPARAMETERS FOUND ===")
print(f"n_estimators = {best_n_estimators}")
print(f"max_depth = {best_max_depth}")
print(f"learning_rate = {best_learning_rate:.4f}")
print(f"subsample = {best_subsample:.4f}")
print(f"colsample_bytree = {best_colsample_bytree:.4f}")
print(f"CV Accuracy = {optimizer.max['target']:.4f}")

# --- STEP 6: Train/Validate Model Once More on a Train/Test Split ---
# Final check on a separate holdout set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=999
)

final_model = XGBClassifier(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    learning_rate=best_learning_rate,
    subsample=best_subsample,
    colsample_bytree=best_colsample_bytree,
    eval_metric='mlogloss',
    random_state=999
)

final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)
test_acc = accuracy(y_pred, y_test)

print("\n=== FINAL EVALUATION ON HOLDOUT TEST SET ===")
print(f"Test Accuracy: {test_acc:.4f}")


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers.scikit_learn'